In [3]:

#Imports the libraries and read the data files

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os, sys
import gensim

from gensim.models import Doc2Vec
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import nltk
from string import punctuation

#from sklearn.cluster import KMeans
#from sklearn import metrics
#import pylab as pl

#def get_doc2vec(filepath, vector_size)

In [6]:
#petition_df = pd.read_csv('./petitions_content_2018.csv', nrows=50000)
petition_df = pd.read_csv('./alimtalk_tempalte_parsed.csv', nrows=500000)
petition_df['parsed_len'] = len(petition)

,temp_id,sender_id,profile_id,content_word,parsed_content,name,category_name,temp_name,target_yn,group_yn
0,NaN,NaN,NaN,계좌번호 가 집중모니터링 계좌로 등록되었습니다,계좌 번호 가 집중 모니터링 계좌 등록되다,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,종통결산액 한도 금액 잔액 금액,종통 결산액 한도 금액 잔액 금액,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,톡톡 구분값 고객 구분값 본인인증 완료,톡톡 구분 값 고객 구분 값 본인 인증 완료,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,님 계좌 원 이자입금되었습니다,님 계좌 원 이자 입금되다,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,님 비밀번호가 월 일 변경되었습니다,님 비밀 번호 월 일 변경되다,NaN,NaN,NaN,NaN,NaN


In [5]:
content = petition_df.loc[petition_df['target_yn'] == "정상-발송됨"]
content = content['parsed_content']
content[31537]

'신유리 고객님 우체국 고객님 님 보내다 택배 등기 번호 을 에 배달 완료 하다 항상 우체국 이용하다 주다 감사합니다 우체국'

In [ ]:
LabeledSentence1 = gensim.models.doc2vec.TaggedDocument
parsed_content = []
j=0
for doc in content:
    if doc:
        parsed_content.append(LabeledSentence1(doc, [j]))
        j+=1
        
print("Number of non-empty vectors: ", j)


In [ ]:
print(parsed_content[1])

In [ ]:
d2v_model = Doc2Vec(parsed_content
                    , vector_size = 1000 # dimension 개수
                    , window = 4
                    , min_count = 20 # Ignores all words with total frequency lower than this.
                    , workers=7
                    , dm = 1
                    , alpha=0.025 # 초기 학습율
                    , min_alpha=0.001)

In [ ]:
d2v_model.similar_by_word

In [ ]:
d2v_model.train(parsed_content
                , total_examples=d2v_model.corpus_count
                , epochs=10
                , start_alpha=0.002
                , end_alpha=-0.016)

In [ ]:
print (d2v_model.docvecs.most_similar(35791, topn=20))

In [ ]:
print(parsed_content[35791])
print(parsed_content[32396])

In [ ]:
# 차원수 확인
print(len(d2v_model.docvecs.vectors_docs[0]))

In [ ]:
kmeans_model = KMeans(n_clusters=20, init='k-means++', max_iter=100)  
X = kmeans_model.fit(d2v_model.docvecs.vectors_docs)
labels=kmeans_model.labels_.tolist()

In [ ]:
l = kmeans_model.fit_predict(d2v_model.docvecs.vectors_docs)
pca = PCA(n_components=2).fit(d2v_model.docvecs.vectors_docs)
datapoint = pca.transform(d2v_model.docvecs.vectors_docs)

In [ ]:
# raw data에 그룹핑 결과 붙이기
petition_df['cluster'] = l


In [ ]:
petition_df

In [ ]:
#petition_df.groupby(['cluster'])['agree_count'].mean()
petition_df.groupby(['cluster'])['agree_count'].count()

In [ ]:
plt.figure
label1 = ["#FFFF00", "#008000", "#0000FF", "#800080"]
color = [label1[i] for i in labels]
plt.scatter(datapoint[:, 0], datapoint[:, 1], c=color)

centroids = kmeans_model.cluster_centers_
centroidpoint = pca.transform(centroids)
plt.scatter(centroidpoint[:, 0], centroidpoint[:, 1], marker='^', s=150, c='#000000')
plt.show()